In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
import pickle
import bz2
from glob import glob

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys, Descriptors, PandasTools, Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.DataStructs import ExplicitBitVect

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, accuracy_score, f1_score,
    roc_auc_score, cohen_kappa_score
)
from sklearn.model_selection import (
    train_test_split, RepeatedStratifiedKFold,
    ShuffleSplit, StratifiedShuffleSplit
)

from standardiser import break_bonds, neutralise, rules, unsalt
from standardiser.utils import StandardiseException, sanity_check

# Optional: untuk autoreload jika di Jupyter
# %reload_ext autoreload
# %autoreload 2

# Suppress warnings
warnings.filterwarnings("ignore")
warnings.warn = lambda *args, **kwargs: None

In [ ]:
import pandas as pd

# Fungsi untuk ubah string ke list of int
def string_to_list(bit_string):
    if isinstance(bit_string, str):
        return list(map(int, bit_string.strip('[]').split(', ')))
    return bit_string

# Load test set dari Excel
test_file = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_Dermal_balanced_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx"
test_df = pd.read_excel(test_file)

# Konversi kolom deskriptor jika masih berupa string
for col in ['Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors']:
    if col in test_df.columns:
        if isinstance(test_df[col].iloc[0], str):
            test_df[col] = test_df[col].apply(string_to_list)

# Tampilkan hasil
print("Test DataFrame:")
print(test_df.head())


In [ ]:
# Melihat nama-nama kolom yang ada di DataFrame
print("Daftar kolom dalam test_df:")
print(test_df.columns.tolist())

In [ ]:
# Cek jumlah NaN sebelum dihapus
nan_before = test_df.isnull().sum().sum()

# Hapus baris yang mengandung NaN
test_df = test_df.dropna()

# Tampilkan informasi jumlah NaN
if nan_before > 0:
    print(f"Total nilai NaN yang dihapus dari test_df: {nan_before}")
else:
    print("Tidak ada nilai NaN yang ditemukan dalam test_df.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Buat salinan kolom Outcome
S = test_df['Outcome'].copy()

# Plot distribusi kelas
fig, ax = plt.subplots()
ax = S.hist(bins=np.arange(-0.5, 5), edgecolor='black')
ax.set_xticks(range(0, 5))
ax.set_xlabel("Outcome Class")
ax.set_ylabel("Count")
ax.set_title("Distribusi Outcome (Test Set)")
plt.show()

# Encoding label
le = LabelEncoder()
outcomes = np.unique(test_df['Outcome'])
le.fit(outcomes)
y = le.transform(test_df['Outcome'])

# Info distribusi
print("Classes                          :", outcomes)
print("Number of cpds in each class     :", np.bincount(y))
print("Total number of cpds             :", len(y))

# Ganti label Outcome menjadi angka (mapping)
S = test_df['Outcome']
info = {}
for i, cls in enumerate(S.unique()):
    info[cls] = i
    S = S.replace(cls, i)

# Optional: simpan mapping info kalau mau pakai nanti
print("Label mapping (kelas → angka):", info)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Ambil label Outcome dari test_df
S = test_df['Outcome'].copy()

# Encode label ke angka
info = {}
for i, cls in enumerate(S.unique()):
    info[cls] = i
    S = S.replace(cls, i)

# Konversi label ke numpy array bertipe int32
y_test = np.int32(S)

# Konversi MACCS, Morgan, dan APF Descriptors ke array numpy
def convert_to_array(desc_list):
    return np.array([eval(desc) if isinstance(desc, str) else desc for desc in desc_list])

x_test_macckeys = convert_to_array(test_df['MACCS_Descriptors'])
x_test_morgan = convert_to_array(test_df['Morgan_Descriptors'])
x_test_apf = convert_to_array(test_df['APF_Descriptors'])  # <-- tambahan APF

# Cek isi
print("Label classes (encoded)       :", info)
print("Jumlah senyawa per kelas      :", np.bincount(y_test))
print("Total jumlah senyawa (test)   :", len(y_test))
print("x_test_macckeys shape         :", x_test_macckeys.shape)
print("x_test_morgan shape           :", x_test_morgan.shape)
print("x_test_apf shape              :", x_test_apf.shape)  # <-- cek APF


In [ ]:
x_rdkitcdk = test_df.drop(columns=['SMILES',
    'Outcome',
    'Morgan_Descriptors',
    'MACCS_Descriptors',
    'APF_Descriptors'])
x_rdkitcdk

In [ ]:
print(x_rdkitcdk)

In [ ]:
x_rdkitcdk  = x_rdkitcdk.apply(lambda row: row.values, axis=1).tolist()

# Add the new column 'rdkit_cdk' to test_df
test_df['rdkit_cdk'] = x_rdkitcdk 

# Display the updated DataFrame
print(test_df)

In [ ]:
y_test = np.int32(S)
x_test_morgan = np.array(list(test_df['Morgan_Descriptors']))
x_test_macckeys = np.array(list(test_df['MACCS_Descriptors']))
x_test_rdkit_cdk = np.array(list(test_df['rdkit_cdk']))
x_test_apf = np.array(list(test_df['APF_Descriptors']))  # <-- tambahan APF


In [ ]:
y_test= np.int32((S))
x_test_rdkit_cdk

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score, classification_report


In [ ]:
y_true = test_df['Outcome'].astype(int)  # Ensure it's of integer type, suitable for metrics calculation


In [ ]:
test_df

In [ ]:
def convert_list_str_to_float(lst):
    return [float(x) for x in lst if x != '' and x is not None]

test_df['rdkit_cdk'] = test_df['rdkit_cdk'].apply(convert_list_str_to_float)

X_rdkitcdk = np.array(test_df['rdkit_cdk'].tolist(), dtype=float)

# EVALUASI DESCRIPTORS

In [ ]:
test_set = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_Dermal_balanced_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import joblib
import itertools
import os
import ast  # untuk konversi string ke list

# ==========================
# Load test set
# ==========================
test_file = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_Dermal_balanced_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx"
test_df = pd.read_excel(test_file)

drop_cols = ['SMILES', 'Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors', 'Outcome']
x_rdkitcdk_test = test_df.drop(columns=drop_cols)
y_true = test_df['Outcome'].astype(int).values

# ==========================
# Helper: konversi string fingerprint ke array
# ==========================
def convert_to_array(series):
    return np.array(series.apply(ast.literal_eval).tolist())

X_test_by_desc = {
    "Morgan": convert_to_array(test_df["Morgan_Descriptors"]),
    "MACCS":  convert_to_array(test_df["MACCS_Descriptors"]),
    "APF":    convert_to_array(test_df["APF_Descriptors"]),
    "Physchem": x_rdkitcdk_test.values  # RDKit+CDK
}

descriptors = ["Morgan", "MACCS", "APF", "Physchem"]
algorithms = ["RF", "XGB", "SVM"]

# ==========================
# Load base models
# ==========================
model_base_dir = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Model"

model_paths = {
    ("Morgan",  "RF"):  os.path.join(model_base_dir, "Dermal_rf_morgan.pkl"),
    ("MACCS",   "RF"):  os.path.join(model_base_dir, "Dermal_rf_macckeys.pkl"),
    ("APF",     "RF"):  os.path.join(model_base_dir, "Dermal_rf_apf.pkl"),
    ("Physchem","RF"):  os.path.join(model_base_dir, "Dermal_rf_rdkitcdk.pkl"),

    ("Morgan",  "XGB"): os.path.join(model_base_dir, "Dermal_xgb_morgan.pkl"),
    ("MACCS",   "XGB"): os.path.join(model_base_dir, "Dermal_xgb_maccs.pkl"),
    ("APF",     "XGB"): os.path.join(model_base_dir, "Dermal_xgb_apf.pkl"),
    ("Physchem","XGB"): os.path.join(model_base_dir, "Dermal_xgb_rdkitcdk.pkl"),

    ("Morgan",  "SVM"): os.path.join(model_base_dir, "Dermal_SVM_Morgan.pkl"),
    ("MACCS",   "SVM"): os.path.join(model_base_dir, "Dermal_SVM_MACCS.pkl"),
    ("APF",     "SVM"): os.path.join(model_base_dir, "Dermal_SVM_APF.pkl"),
    ("Physchem","SVM"): os.path.join(model_base_dir, "Dermal_svm_rdkitcdk.pkl"),
}

models = {}
for (desc, algo), path in model_paths.items():
    models[(desc, algo)] = joblib.load(path)

print("Semua base QSAR models berhasil dimuat.\n")

# ==========================
# Load Sm (10-fold scaffold-CV) untuk weighting
# ==========================

sm_files = {
    ("Morgan",  "RF"):  os.path.join(model_base_dir, "Sm_Morgan_RF.csv"),
    ("MACCS",   "RF"):  os.path.join(model_base_dir, "Sm_MACCS_RF.csv"),
    ("APF",     "RF"):  os.path.join(model_base_dir, "Sm_APF_RF.csv"),
    ("Physchem","RF"):  os.path.join(model_base_dir, "Sm_Physchem_RF.csv"),

    ("Morgan",  "XGB"): os.path.join(model_base_dir, "Sm_Morgan_XGB.csv"),
    ("MACCS",   "XGB"): os.path.join(model_base_dir, "Sm_MACCS_XGB.csv"),
    ("APF",     "XGB"): os.path.join(model_base_dir, "Sm_APF_XGB.csv"),
    ("Physchem","XGB"): os.path.join(model_base_dir, "Sm_Physchem_XGB.csv"),

    ("Morgan",  "SVM"): os.path.join(model_base_dir, "Sm_Morgan_SVM.csv"),
    ("MACCS",   "SVM"): os.path.join(model_base_dir, "Sm_MACCS_SVM.csv"),
    ("APF",     "SVM"): os.path.join(model_base_dir, "Sm_APF_SVM.csv"),
    ("Physchem","SVM"): os.path.join(model_base_dir, "Sm_Physchem_SVM.csv"),
}

Sm = {}
for key, path in sm_files.items():
    df_sm = pd.read_csv(path)
    Sm[key] = float(df_sm["Sm"].values[0])

# ==========================
# Bootstrap metrics (median ± half-range)
# ==========================
def bootstrap_metrics_simple(probs, y_true, n_bootstrap=1000, seed=42):
    rng = np.random.default_rng(seed)
    y_true = np.asarray(y_true)
    probs = np.asarray(probs)
    preds = (probs >= 0.5).astype(int)
    n = len(y_true)

    accs, sens_list, spec_list, aucs = [], [], [], []
    tn_list, fp_list, fn_list, tp_list = [], [], [], []

    for _ in range(n_bootstrap):
        idx = rng.choice(n, size=n, replace=True)
        y_b = y_true[idx]
        p_b = probs[idx]
        pred_b = preds[idx]

        tn, fp, fn, tp = confusion_matrix(y_b, pred_b).ravel()
        tn_list.append(tn)
        fp_list.append(fp)
        fn_list.append(fn)
        tp_list.append(tp)

        accs.append(accuracy_score(y_b, pred_b))
        sens = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        sens_list.append(sens)
        spec_list.append(spec)
        try:
            aucs.append(roc_auc_score(y_b, p_b))
        except ValueError:
            aucs.append(np.nan)

    def summarize(values):
        values = np.array(values, dtype=float)
        med = np.nanmedian(values)
        lo = np.nanmin(values)
        hi = np.nanmax(values)
        err = (hi - lo) / 2.0
        return f"{med:.3f} ± {err:.3f}"

    metrics = {
        "Accuracy":   summarize(accs),
        "Sensitivity":summarize(sens_list),
        "Specificity":summarize(spec_list),
        "AUC":        summarize(aucs),
        "TN": int(np.mean(tn_list)),
        "FP": int(np.mean(fp_list)),
        "FN": int(np.mean(fn_list)),
        "TP": int(np.mean(tp_list)),
    }
    return metrics

# ==========================
# 1) Performansi individual base-model
# ==========================
results = []

for desc in descriptors:
    X_desc = X_test_by_desc[desc]
    for algo in algorithms:
        model = models[(desc, algo)]
        probs = model.predict_proba(X_desc)[:, 1]
        m = bootstrap_metrics_simple(probs, y_true)
        m["Type"] = "Individual"
        m["Descriptor"] = desc
        m["Algorithm"] = algo
        m["Name"] = f"{desc}-{algo}"
        results.append(m)

# ==========================
# 2) Consensus per descriptor (across algorithms) dengan bobot Sm
# ==========================
descriptor_consensus_probs = {}

for desc in descriptors:
    X_desc = X_test_by_desc[desc]

    # kumpulkan prob per algoritme + Sm
    probs_alg = {}
    sm_alg = []
    for algo in algorithms:
        key = (desc, algo)
        model = models[key]
        probs_alg[algo] = model.predict_proba(X_desc)[:, 1]
        sm_alg.append(Sm[key])

    sm_alg = np.array(sm_alg, dtype=float)
    w_alg = sm_alg / sm_alg.sum()  # normalisasi bobot

    # urutkan sesuai algorithms agar konsisten
    prob_matrix = np.vstack([probs_alg[algo] for algo in algorithms])  # shape: (3, n_samples)
    cons_probs = np.average(prob_matrix, axis=0, weights=w_alg)
    descriptor_consensus_probs[desc] = cons_probs

    m = bootstrap_metrics_simple(cons_probs, y_true)
    m["Type"] = "DescriptorConsensus"
    m["Descriptor"] = desc
    m["Algorithm"] = "RF+XGB+SVM"
    m["Name"] = f"{desc}_QSAR_consensus"
    results.append(m)

# ==========================
# 3) Full consensus QSAR across descriptors (weighted by Sm descriptor-level)
# ==========================
# contoh: weight descriptor-level = rata-rata Sm semua algoritme untuk descriptor tsb
Sm_desc = {}
for desc in descriptors:
    sm_list = [Sm[(desc, algo)] for algo in algorithms]
    Sm_desc[desc] = float(np.mean(sm_list))

Sm_desc_arr = np.array([Sm_desc[d] for d in descriptors], dtype=float)
w_desc = Sm_desc_arr / Sm_desc_arr.sum()

prob_matrix_desc = np.vstack([descriptor_consensus_probs[d] for d in descriptors])  # (4, n_samples)
full_qsar_probs = np.average(prob_matrix_desc, axis=0, weights=w_desc)

m_full = bootstrap_metrics_simple(full_qsar_probs, y_true)
m_full["Type"] = "FullQSARConsensus"
m_full["Descriptor"] = "All"
m_full["Algorithm"] = "QSAR_consensus"
m_full["Name"] = "Full_QSAR_mfCoQ_component"
results.append(m_full)

# ==========================
# Simpan hasil ke Excel
# ==========================
results_df = pd.DataFrame(results)

# ekstrak nilai AUC median untuk sorting
results_df["AUC_val"] = results_df["AUC"].str.extract(r"([0-9.]+)").astype(float)
results_df = results_df.sort_values(by="AUC_val", ascending=False).drop(columns=["AUC_val"])

cols_order = ["Type", "Name", "Descriptor", "Algorithm", "AUC", "Accuracy",
              "Sensitivity", "Specificity", "TN", "FP", "FN", "TP"]
results_df = results_df[cols_order]

save_path = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Evaluation\Evaluation_QSAR_with_Sm_weighting.xlsx"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
results_df.to_excel(save_path, index=False)

print(f"Hasil evaluasi QSAR (individual + descriptor consensus + full QSAR) disimpan ke:\n{save_path}")


# Without CI 95%

In [ ]:
# ================================
# 라이브러리 임포트
# ================================
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import joblib
import os
import ast  # 문자열을 리스트/배열로 변환할 때 사용

# ================================
# 테스트셋 경로 설정
# ================================
test_files = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Test_set_Dermal_balanced_with_fingerprints_sorted_with_RDKit_and_CDK_features.xlsx"
if isinstance(test_files, str):
    test_files = [test_files]

# ================================
# 모델 불러오기 (base models)
# ================================
model_base_dir = r"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Model"

descriptors = ["Morgan", "MACCS", "APF", "Physchem"]
algorithms = ["RF", "XGB", "SVM"]

model_paths = {
    ("Morgan",  "SVM"): os.path.join(model_base_dir, "Dermal_SVM_Morgan.pkl"),
    ("MACCS",   "SVM"): os.path.join(model_base_dir, "Dermal_SVM_MACCS.pkl"),
    ("APF",     "SVM"): os.path.join(model_base_dir, "Dermal_SVM_APF.pkl"),
    ("Physchem","SVM"): os.path.join(model_base_dir, "Dermal_svm_rdkitcdk.pkl"),

    ("Morgan",  "RF"):  os.path.join(model_base_dir, "Dermal_rf_morgan.pkl"),
    ("MACCS",   "RF"):  os.path.join(model_base_dir, "Dermal_rf_macckeys.pkl"),
    ("APF",     "RF"):  os.path.join(model_base_dir, "Dermal_rf_apf.pkl"),
    ("Physchem","RF"):  os.path.join(model_base_dir, "Dermal_rf_rdkitcdk.pkl"),

    ("Morgan",  "XGB"): os.path.join(model_base_dir, "Dermal_xgb_morgan.pkl"),
    ("MACCS",   "XGB"): os.path.join(model_base_dir, "Dermal_xgb_maccs.pkl"),
    ("APF",     "XGB"): os.path.join(model_base_dir, "Dermal_xgb_apf.pkl"),
    ("Physchem","XGB"): os.path.join(model_base_dir, "Dermal_xgb_rdkitcdk.pkl"),
}

loaded_models = {}
for key, path in model_paths.items():
    loaded_models[key] = joblib.load(path)
print("✅ 모든 base QSAR 모델이 성공적으로 로드되었습니다.\n")

# ================================
# Sm (10-fold scaffold-CV score) 로딩
# ================================
sm_files = {
    ("Morgan",  "RF"):  os.path.join(model_base_dir, "Sm_Morgan_RF.csv"),
    ("MACCS",   "RF"):  os.path.join(model_base_dir, "Sm_MACCS_RF.csv"),
    ("APF",     "RF"):  os.path.join(model_base_dir, "Sm_APF_RF.csv"),
    ("Physchem","RF"):  os.path.join(model_base_dir, "Sm_Physchem_RF.csv"),

    ("Morgan",  "XGB"): os.path.join(model_base_dir, "Sm_Morgan_XGB.csv"),
    ("MACCS",   "XGB"): os.path.join(model_base_dir, "Sm_MACCS_XGB.csv"),
    ("APF",     "XGB"): os.path.join(model_base_dir, "Sm_APF_XGB.csv"),
    ("Physchem","XGB"): os.path.join(model_base_dir, "Sm_Physchem_XGB.csv"),

    ("Morgan",  "SVM"): os.path.join(model_base_dir, "Sm_Morgan_SVM.csv"),
    ("MACCS",   "SVM"): os.path.join(model_base_dir, "Sm_MACCS_SVM.csv"),
    ("APF",     "SVM"): os.path.join(model_base_dir, "Sm_APF_SVM.csv"),
    ("Physchem","SVM"): os.path.join(model_base_dir, "Sm_Physchem_SVM.csv"),
}

Sm = {}
for key, path in sm_files.items():
    sm_df = pd.read_csv(path)
    Sm[key] = float(sm_df["Sm"].values[0])

# ================================
# 평가 지표 계산 함수
# ================================
def compute_metrics(probs, y_true):
    preds = (probs > 0.5).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    acc = accuracy_score(y_true, preds)
    sen = tp / (tp + fn) if (tp + fn) > 0 else 0
    spe = tn / (tn + fp) if (tn + fp) > 0 else 0
    try:
        auc = roc_auc_score(y_true, probs)
    except:
        auc = np.nan
    return {
        'Accuracy': round(acc, 3),
        'Sensitivity': round(sen, 3),
        'Specificity': round(spe, 3),
        'AUC': round(auc, 3),
        'TN': tn, 'FP': fp, 'FN': fn, 'TP': tp
    }

def convert_to_array(series):
    return np.array(series.apply(ast.literal_eval).tolist())

# ================================
# 테스트셋 처리 루프
# ================================
for test_file in test_files:
    print(f"🔍 파일 처리 중: {test_file}")
    test_df = pd.read_excel(test_file)

    drop_cols = ['SMILES', 'Morgan_Descriptors', 'MACCS_Descriptors', 'APF_Descriptors', 'Outcome']
    x_rdkitcdk_test = test_df.drop(columns=drop_cols)
    y_true = test_df['Outcome'].astype(int).values

    X_test_by_desc = {
        "Morgan":   convert_to_array(test_df["Morgan_Descriptors"]),
        "MACCS":    convert_to_array(test_df["MACCS_Descriptors"]),
        "APF":      convert_to_array(test_df["APF_Descriptors"]),
        "Physchem": x_rdkitcdk_test.values
    }

    results_list = []

    # 1) Individual base models
    for desc in descriptors:
        X_desc = X_test_by_desc[desc]
        for algo in algorithms:
            model = loaded_models[(desc, algo)]
            probs = model.predict_proba(X_desc)[:, 1]
            metrics = compute_metrics(probs, y_true)
            metrics['Type'] = "Individual"
            metrics['Combination'] = f"{desc}-{algo}"
            results_list.append(metrics)

    # 2) Consensus per descriptor (RF+XGB+SVM, dibobot Sm)
    desc_cons_probs = {}
    for desc in descriptors:
        X_desc = X_test_by_desc[desc]

        probs_alg = []
        sm_alg = []
        for algo in algorithms:
            model = loaded_models[(desc, algo)]
            probs_alg.append(model.predict_proba(X_desc)[:, 1])
            sm_alg.append(Sm[(desc, algo)])

        probs_alg = np.vstack(probs_alg)   # shape (3, n_samples)
        sm_alg = np.array(sm_alg, dtype=float)
        w_alg = sm_alg / sm_alg.sum()

        cons_probs_desc = np.average(probs_alg, axis=0, weights=w_alg)
        desc_cons_probs[desc] = cons_probs_desc

        metrics = compute_metrics(cons_probs_desc, y_true)
        metrics['Type'] = "DescriptorConsensus"
        metrics['Combination'] = f"{desc}_QSAR_consensus(RF+XGB+SVM)"
        results_list.append(metrics)

    # 3) Full QSAR consensus across descriptors (dibobot Sm descriptor-level)
    Sm_desc = {}
    for desc in descriptors:
        Sm_desc[desc] = float(np.mean([Sm[(desc, algo)] for algo in algorithms]))
    w_desc = np.array([Sm_desc[d] for d in descriptors], dtype=float)
    w_desc = w_desc / w_desc.sum()

    probs_desc_mat = np.vstack([desc_cons_probs[d] for d in descriptors])  # (4, n_samples)
    full_qsar_probs = np.average(probs_desc_mat, axis=0, weights=w_desc)

    metrics = compute_metrics(full_qsar_probs, y_true)
    metrics['Type'] = "FullQSARConsensus"
    metrics['Combination'] = "QSAR_full_mfCoQ_component"
    results_list.append(metrics)

    # ================================
    # 결과 저장
    # ================================
    metrics_df = pd.DataFrame(results_list).sort_values(by="AUC", ascending=False)
    metrics_df = metrics_df[['Type', 'Combination', 'AUC', 'Accuracy', 'Sensitivity', 'Specificity', 'TN', 'FP', 'FN', 'TP']]

    set_name = os.path.splitext(os.path.basename(test_file))[0]
    save_path = fr"C:\Fauzan\Manuskrip QSAR 1\Major Revision\Acute Dermal Toxicity (manual split)\Evaluation\NoCI95_Evaluation_{set_name}_QSAR_with_Sm_weighting.xlsx"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    metrics_df.to_excel(save_path, index=False)

    print(f"✅ {set_name}의 QSAR 개별/컨센서스 결과({len(metrics_df)})가 저장되었습니다:\n   {save_path}\n")
